## Dipole action comparison between qml's `dipole_moment` and Stepan's `dipole_action`

This script was written to demonstrate a discrepancy between a dipole operator implementation in PennyLane `dipole_moment` and the `dipole_action` implementation in the xas_simulation repository. We use a simple $H_2$ molecule aligned in the $z$ axis and calculate the dipole action on the ground state of that molecule to produce $m_\rho |I\rangle$. This is done in both ways to compare the output state.

First, create initial state $|I\rangle$ with PySCF CASCI (using CASCI here because it is what is used in the repo).

In [ ]:
from pyscf import gto, scf, mcscf
import numpy as np
from pyscf.fci.cistring import addrs2str
from scipy.sparse import coo_matrix
import pennylane as qml

# Create a H_2 mol object
r = 0.71
symbols  = ['H', 'H']
geometry = np.array([[0.0, 0.0, -r/2],
                      [0.0, 0.0, r/2]])
mol = gto.Mole(atom=zip(symbols, geometry), basis='631g')
mol.build()

# get MOs
hf = scf.RHF(mol)
hf.run(verbose=0)

norb = hf.mol.nao
nelec = hf.mol.nelectron

# make CASCI object
mycasci = mcscf.CASCI(hf, norb, nelec)
mycasci.run(verbose=0)

ncas_a = mycasci.ncas
ncas_b = ncas_a
nelecas_a, nelecas_b = mycasci.nelecas

# ground state
casci_state = mycasci.ci

# filter out small values based on preset tolerance to save more memory
casci_state[abs(casci_state) < 1e-6] = 0
print("|I> CASCI ground_state \n", casci_state)

sparse_cascimatr = coo_matrix(casci_state, shape=np.shape(casci_state), \
                                                        dtype=float )
row, col, dat = sparse_cascimatr.row, sparse_cascimatr.col, \
                                                sparse_cascimatr.data
# round output
dat = np.round(dat, 5)

## turn indices into strings
strs_row = addrs2str(ncas_a, nelecas_a, row)
strs_col = addrs2str(ncas_b, nelecas_b, col)

## create the FCI matrix as a dict
wf_dict = dict(zip(list(zip(strs_row, strs_col)), dat))

print("\n|I> wf_dict")
for k, v in wf_dict.items():
    print(k, v)

|I> CASCI ground_state 
 [[ 0.99327285  0.          0.00485881  0.        ]
 [ 0.         -0.07091585  0.         -0.04359152]
 [ 0.00485881  0.         -0.05193492  0.        ]
 [ 0.         -0.04359152  0.         -0.04283796]]

|I> wf_dict
(1, 1) 0.99327
(1, 4) 0.00486
(2, 2) -0.07092
(2, 8) -0.04359
(4, 1) 0.00486
(4, 4) -0.05193
(8, 2) -0.04359
(8, 8) -0.04284


Now apply `dipole_action` from xas_simulation repo. Below is a minimal implementation of that function.

In [ ]:
import itertools

def _collect_terms(messy_dict):
    simpl_dict = {}
    for _, compdict in messy_dict.items():
        for key, val in compdict.items():
            if key in simpl_dict.keys():
                simpl_dict[key] += val
            else:
                simpl_dict[key] = val
    return simpl_dict

# The following has been simpified from repo to just act on the full orbital space
def dipole_action(hf, norbs, wf_dict, rho=2, dipole_cutoff=1e-6):
    """Applies the dipole operator in Cartesian direction rho to wavefunction 
    dictionary. Ignores coupling elements below dipole_cutoff."""

    dip_ints = hf.mol.intor('int1e_r_cart', comp=3) 
    mo_coeffs = hf.mo_coeff

    # get the dipole integrals in MO basis,
    dip_ints = np.einsum('ik,xkl,lj->xij', mo_coeffs.T, dip_ints, mo_coeffs)

    N = norbs

    all_dicts_rho = []

    # run over each determinant in |I>, apply all dipole elements
    for key, val in wf_dict.items():
        dipole_dict = {}

        # for a determinant "key", run over all possible dipole terms
        for (i,j) in list(itertools.product(range(dip_ints.shape[1]),\
                                            range(dip_ints.shape[2]))):
            if abs(dip_ints[rho,i,j]) < dipole_cutoff:
                continue
            rev_i, rev_j = (N-1) - i, (N-1) - j
            # account for both spins -- dipole the same in both
            for spin in [0, 1]:
                if spin == 0:
                    # apply c^dag_i c_j if occupancy j = 1 and occupancy i = 0
                    occ_str_a = format(key[0], f"#0{N+2}b")
                    if (occ_str_a[2+rev_j] == "1") & \
                        (occ_str_a[2+rev_i] == "0"):
                        newkey_up = (key[0] | (1<<i)) & ~(1<<j)
                        newkey_do = key[1]
                        sign_destroyer = occ_str_a[2+rev_j+1:].count("1")
                        occ_str_a_new = format(newkey_up, f"#0{N+2}b")
                        sign_creator = occ_str_a_new[2+rev_i:].count("1")
                        total_sign = (-1)**(sign_creator + sign_destroyer + 1)
                    # apply c^dag_i c_i -- number operator
                    elif (rev_i == rev_j) & (format(key[0], f"#0{N+2}b")[2+rev_j] == "1"):
                        newkey_up = key[0]
                        newkey_do = key[1]
                        total_sign = 1
                    # if j = 0 or i != 0, that goes to zero
                    else:
                        continue
                # same as above
                elif spin == 1:
                    # apply c^dag_i c_j if occupancy j = 1 and occupancy i = 0
                    occ_str_b = format(key[1], f"#0{N+2}b")
                    if (occ_str_b[2+rev_j] == "1") & \
                        (occ_str_b[2+rev_i] == "0"):
                        newkey_up = key[0]
                        newkey_do = (key[1] | (1<<i)) & ~(1<<j)
                        sign_destroyer = occ_str_b[2+rev_j+1:].count("1")
                        occ_str_b_new = format(newkey_do, f"#0{N+2}b")
                        sign_creator = occ_str_b_new[2+rev_i:].count("1")
                        total_sign = (-1)**(sign_creator + sign_destroyer + 1)
                    elif (rev_i == rev_j) & (format(key[1], f"#0{N+2}b")[2+rev_j] == "1"):
                        newkey_up = key[0]
                        newkey_do = key[1]
                        total_sign = 1
                    else:
                        continue
                # add coefficient == dipole integral value * Slater det coeff
                if (newkey_up, newkey_do) in dipole_dict.keys():
                    dipole_dict[(newkey_up, newkey_do)] += total_sign * dip_ints[rho,i,j] * val
                else:
                    dipole_dict[(newkey_up, newkey_do)] = total_sign * dip_ints[rho,i,j] * val

        # once built, add the dipole dict to the correct axis
        all_dicts_rho[key] = dipole_dict
                    
        # simplify to obtain final state 
        dipole_rho = _collect_terms(all_dicts_rho)

        # normalize the states
        norm_rho = np.sqrt(np.sum(np.array(list(dipole_rho.values())) ** 2))
        dipole_rho = {key: value / norm_rho for (key, value) in dipole_rho.items()}

    return dipole_rho, norm_rho

In [ ]:
from scipy import sparse

rho = 2
m_rho_wf_dict, _ = dipole_action(hf, norb, wf_dict, rho=rho)

for key, val in m_rho_wf_dict.items():
    m_rho_wf_dict[key] = round(val, 5)

print("m_rho_wf_dict")
for k, v in m_rho_wf_dict.items():
    print(k, v)

m_rho_wf = qml.qchem.convert._wfdict_to_statevector(m_rho_wf_dict, norb)
m_rho_wf = np.round(m_rho_wf, 5)

sparse_m_rho_wf = sparse.dok_array(m_rho_wf, shape=m_rho_wf.shape)

print("\ndipole_action m_rho_wf")
for k, v in sparse_m_rho_wf.items():
    print(k, v)

m_rho_wf_dict
(2, 1) 0.69026
(1, 2) 0.69026
(8, 1) 0.13271
(1, 8) 0.13271
(2, 4) 0.07025
(8, 4) -0.03161
(4, 2) 0.07025
(4, 8) -0.03161

dipole_action m_rho_wf
6 (-0.03161+0j)
9 (-0.03161+0j)
24 (0.07025+0j)
36 (0.07025+0j)
66 (0.13271+0j)
96 (0.69026+0j)
129 (0.13271+0j)
144 (0.69026+0j)


## PennyLane $H_2$

In [55]:
a0 = 0.529177
coordinates_au = geometry.flatten()/a0

d_dof = qml.qchem.dipole_of(symbols, coordinates_au, basis='6-31g')
dip_matrix = np.array(qml.matrix(d_dof[rho]))

from pyscf import ci
myci = ci.CISD(hf).run()
wf = qml.qchem.import_state(myci, tol=1e-5)

# wf = qml.qchem.convert._wfdict_to_statevector(wf_dict, ncas)  # CASCI

qml_m_rho_wf = dip_matrix.dot(wf)

# normalize and round
qml_m_rho_wf = qml_m_rho_wf/np.sqrt(np.dot(qml_m_rho_wf, qml_m_rho_wf))
qml_m_rho_wf = np.round(qml_m_rho_wf, 5)

sparse_qml_m_rho_wf = sparse.dok_array(qml_m_rho_wf, shape=qml_m_rho_wf.shape)

print("\nPennyLane\t dipole_action")
for k, v in sparse_qml_m_rho_wf.items():
    print(k, sparse_qml_m_rho_wf[k], sparse_m_rho_wf[k])

E(RCISD) = -1.150773565559498  E_corr = -0.02425163008954728

PennyLane	 dipole_action
6 (0.03161+0j) (-0.03161+0j)
9 (-0.03161+0j) (-0.03161+0j)
24 (-0.07025+0j) (0.07025+0j)
36 (0.07025+0j) (0.07025+0j)
66 (-0.13271+0j) (0.13271+0j)
96 (-0.69026+0j) (0.69026+0j)
129 (0.13271+0j) (0.13271+0j)
144 (0.69026+0j) (0.69026+0j)


![alt text](<figures_h2_qml/spectrum/2025-06-05 17:03:00_spectra.png>)